### TP 3 EX 2
Grupo 5:
     
     Breno Fernando Guerra Marrão A97768
     
     Tales André Rovaris Machado A96314

```Python
{ x != (0,0,0,0) }
1:     x[0] = x[-1] || x[0]⊕x[-1]    
2:     x[1] = x[0] || x[1]⊕x[0]  
3:     x[2] = x[1] || x[2]⊕x[1]   
4:     x[3] = x[2] || x[3]⊕x[2]          
5: ERROR
```
Para modelar este programa como um SFOTS teremos o conjunto $\mathsf{X}$ de variáveis do estado dado pela lista `['x','pc']`, e definimos a função
`genState` que recebe a lista com o nome das variáveis do estado, uma etiqueta e um inteiro, e cria a i-ésima cópia das variáveis do estado para essa etiqueta. As variáveis lógicas começam sempre com o nome de base das variáveis dos estado, seguido do separador `!`.

## Inicialização
Usamos as bibliotecas pysmt para resovler o problema de alocação proposto 

In [2]:
from pysmt.shortcuts import *
from pysmt.typing import INT
import numpy
import itertools 


Para modelar este programa como um SFOTS teremos o conjunto $\mathsf{X}$ de variáveis do estado dado pela lista `['x','pc']`, e definimos a função
`genState` que recebe a lista com o nome das variáveis do estado, uma etiqueta e um inteiro, e cria a i-ésima cópia das variáveis do estado para essa etiqueta. As variáveis lógicas começam sempre com o nome de base das variáveis dos estado, seguido do separador `!`.

In [ ]:
def genState(vars,s,i):
    state = {}
    for v in vars:
        state[v] = Symbol(v+'!'+s+str(i),INT)
    return state

Defina as seguintes funções para completar a modelação deste programa:
- `init1` dado um estado do programa (um dicionário de variáveis), devolve um predicado do pySMT que testa se esse estado é um possível estado inicial do programa.
- `error1` dado um  estado do programa, devolve um predicado do pySMT que testa se esse estado é um possível estado de erro do programa.
- `trans1` que, dados dois estados do programa, devolve um predicado do pySMT que testa se é possível transitar de algum estado para outro

In [2]:
def init1(state,p):
    t1 = Equals(state['pc'],Int(0))
    t2 = And(Equals(state['s'][i],numpy.random.choice([Int(0),Int(1)], p=[p,1-p]))for i in state if i != 'pc')
    t3 = Not(And(Equals(state['a'], Int(0)),Equals(state['b'], Int(0)),Equals(state['c'],
                                                                             Int(0)),Equals(state['d'], Int(0))))
    op = [numpy.random.choice([Int(0),Int(1)], p=[p,1-p]) for i in range(3)]
    return (And(t1,t2,t3),op)

def error1(state):
    return Not(And(Equals(state['a'], Int(0)),Equals(state['b'], Int(0)),Equals(state['c'],
                                                                             Int(0)),Equals(state['d'], Int(0))))
def trans1(curr, prox,op):
    t0 = And(Equals(prox['a'],Ite(op[0],Not(curr['c']),Xor(curr['a'], curr['c']))),
                                       Equals(curr['pc'],Int(0)),Equals(prox['pc'],Int(1))
                         ,Equals(curr['b'],prox['b']),Equals(curr['c'],prox['c'],Equals(curr['d'],prox['d'])))
    t1 = And(Equals(prox['b'],Ite(op[1],Not(curr['a']),Xor(curr['a'], curr['b']))),
                                       Equals(curr['pc'],Int(1)),Equals(prox['pc'],Int(2))
                         ,Equals(curr['a'],prox['a']),Equals(curr['c'],prox['c'],Equals(curr['d'],prox['d'])))
    t2 = And(Equals(prox['d'],Ite(op[2],Not(curr['b']),Xor(curr['d'], curr['b']))),
                                       Equals(curr['pc'],Int(2)),Equals(prox['pc'],Int(3))
                         ,Equals(curr['a'],prox['a']),Equals(curr['c'],prox['c'],Equals(curr['b'],prox['b'])))
    t3 = And(Equals(prox['c'],Ite(op[2],Not(curr['d']),Xor(curr['d'], curr['c']))),
                                       Equals(curr['pc'],Int(3)),Equals(prox['pc'],Int(0))
                         ,Equals(curr['a'],prox['a']),Equals(curr['d'],prox['d'],Equals(curr['b'],prox['b'])))
    return Or(t0,t1,t2,t3)        